# Autopilot Experiment with SageMaker Studio UI

This notebook works well with the `Python 3 (Data Science)` kernel on SageMaker Studio.

---

---

## Contents

1. [Introduction](#Introduction)
1. [Setup](#Setup)
1. [AutoPilot Experiment using SageMaker Studio UI](#AutoPilot-Experiment-using-SageMaker-Studio-UI)
 * [Open Amazon SageMaker Studio](#Open-Amazon-SageMaker-Studio)
 * [Create Autopilot Experiment](#Create-Autopilot-Experiment-Job)
 * [Enter information for the AutoPilot Job](#Enter-information-for-the-AutoPilot-Job)
 * [View Autopilot Experiment Job ](#View-Autopilot-Experiment-Job)
 * [Test Deployed Model](#Test-Deployed-Model)
1. [Clean Up](#Cleanup)

## Setup

Retrieve shared variables created by [01_sagemaker_autopilot_setup.ipynb](./01_sagemaker_autopilot_setup.ipynb) notebook and list out the S3 URIs to prepare Autopilot experiment.

In [2]:
#cell - 01
%store -r train_data_s3_path
%store -r test_file_with_label
%store -r lab_ap_prefix
%store -r bucket

try:
  train_data_s3_path
except NameError:
    raise ValueError("Training dataset S3 URI is missing, please execute the data preparation notebook!")

> **__NOTE__** Please note down output of the variables below (cells 02 and 03).
* `train_data_s3_path` for training data input.
* `using_studio_ui_output_path` for Autopilot experiment output.

In [5]:
#cell - 02
train_data_s3_path

's3://sagemaker-ap-southeast-2-783193689466/mlu-workshop/direct-marketing/autopilot/train/train_data.csv'

In [4]:
#cell - 03
using_studio_ui_output_path = f"s3://{bucket}/{lab_ap_prefix}/using-studio-ui-output"
using_studio_ui_output_path

's3://sagemaker-ap-southeast-2-783193689466/mlu-workshop/direct-marketing/autopilot/using-studio-ui-output'

## Create Autopilot Experiment with SageMaker Studio UI

> Now, follow the instructions outlined in Step 4 in <a href="https://github.com/tom5610/mlu-workshop/tree/main/1.autopilot#Using-SageMaker-Studio-UI" target="_blank">UI tutorial</a> to create Autopilot Experiment. 

> Once model has been deployed, proceed with the cells below to test the deployed model.

## Test Deployed Model

> Please ensure that you've deployed the Autopilot trained model while creating the experiment.

View the split test dataset with label.

In [ ]:
#cell - 04
import pandas as pd

column_label = 'y'

test_data = pd.read_csv(test_file_with_label)
columns = test_data.columns.tolist()
columns.remove(column_label)
columns.insert(0, column_label)

# list the label as first column so that you can verify the prediction result easier
test_data[columns]

In [ ]:
#cell - 05
X_test_numpy = test_data.drop(["y"], axis=1).values

set the endpoint name, if you are using something different from `dm-autopilot-experiment`, please update the value below:

In [ ]:
#cell - 06
endpoint_name = 'dm-autopilot-experiment02'

In [ ]:
#cell - 07
import boto3

runtime = boto3.client('sagemaker-runtime')


def predict(payload):
    response = runtime.invoke_endpoint(EndpointName=endpoint_name,
                                           ContentType='text/csv',
                                           Body=payload)
    print(response)
    result = response['Body'].read().decode('utf-8').strip()
    pred, pred_probability = result.split(',')
    print(f"Prediction result: {pred} with probability: {pred_probability}")
    return pred, pred_probability

In [ ]:
#cell - 08
# update the index to test on different row in the test dataset.
index = 15

label = test_data.iloc[index]['y']
print(f"Row index {index} with Label: {label}")

payload = ','.join(X_test_numpy[index].astype(str).tolist())
predict(payload)

### Cleanup

It's generally a good practice to deactivate all endpoints which are not in use.

Please uncomment the following lines and run the cell in order to deactivate the endpoint that were created before.

In [ ]:
# sm_client = boto3.client('sagemaker')
# sm_client.delete_endpoint(EndpointName=endpoint_name)